In [1]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
  The repository located at h2o-release.s3.amazonaws.com is not a trusted or secure host and is being ignored. If this repository is available via HTTPS we recommend you use HTTPS instead, otherwise you may silence this warning and allow it anyway with '--trusted-host h2o-release.s3.amazonaws.com'.
    100% |████████████████████████████████| 120.9MB 199kB/s 
  The repository located at h2o-release.s3.amazonaws.com is not a trusted or secure host and is being ignored. If this repository is available via HTTPS we recommend you use HTTPS instead, otherwise you may silence this warning and allow it anyway with '--trusted-host h2o-release.s3.amazonaws.com'.
  Stored in directory: /root/.cache/pip/wheels/f4/21/59/e953dbf8206d5061ce1346e119dad9ffafd7f88011430682ba
Successfully built h2o


In [59]:
import h2o
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,39 mins 21 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_unknownUser_bzwy0k
H2O cluster total nodes:,1
H2O cluster free memory:,2.976 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


In [0]:
from sklearn.metrics import accuracy_score

In [0]:
scores= pd.read_csv('combined_GE.csv')

In [62]:
scores.head(5)

,Unnamed: 0,AWSSentiment,AWSMixed,AWSNegative,AWSNeutral,AWSPositive,GCPmagnitude,GCPscore,IBMlabel,IBMscore,id,Azurescore,realSentiment,Text
0,1,POSITIVE,0.012899,0.001309,0.180401,0.805391,1.0,0.2,positive,0.871771,1,0.812409,neutral,"Good day, ladies and gentlemen and welcome to ..."
1,2,NEUTRAL,0.002106,0.026452,0.952022,0.019421,0.0,0.0,positive,0.607113,2,0.823700,neutral,I would now like to turn the program over to y...
2,3,POSITIVE,0.003473,0.001572,0.210365,0.784590,1.6,0.4,positive,0.572983,3,0.762308,neutral,In the spirit of providing you with all the in...
3,4,NEGATIVE,0.039225,0.559341,0.396765,0.004668,0.0,0.0,positive,0.547195,4,0.500000,neutral,"Well, Jeff what I was referring to there was t..."
4,5,NEUTRAL,0.030986,0.167251,0.796661,0.005102,1.9,0.0,negative,-0.552928,5,0.500000,neutral,Morning. Maybe as the first question on Power....


In [0]:
coloumns=['AWSMixed','AWSNegative','AWSNeutral','AWSPositive','GCPscore','IBMscore','Azurescore']

In [0]:
X=np.asarray(scores[coloumns])
y=np.asarray(scores['realSentiment'])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [66]:
x_train.shape

(123, 7)

In [0]:
aml = H2OAutoML(max_models=20, seed=1)

In [0]:
df = pd.DataFrame(x_train)
df['realSentiment'] = y_train

In [69]:
h2o_data = h2o.import_file('combined_GE.csv')

predictors = ['AWSMixed','AWSNegative','AWSNeutral','AWSPositive','GCPscore','IBMscore','Azurescore']
response = "realSentiment"

train, valid = h2o_data.split_frame(ratios = [.8], seed = 11)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [70]:
aml.train(x=predictors, y=response, training_frame=h2o_data)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [71]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
DeepLearning_1_AutoML_20190324_043515,0.505613,1.2934,0.508661,0.258736
DeepLearning_grid_1_AutoML_20190324_043515_model_2,0.585335,1.77334,0.512546,0.262703
DeepLearning_grid_1_AutoML_20190324_043515_model_1,0.588589,1.68935,0.490455,0.240546
DeepLearning_grid_1_AutoML_20190324_043515_model_3,0.595095,1.29829,0.478349,0.228817
GBM_grid_1_AutoML_20190324_043515_model_2,0.608037,1.71028,0.551052,0.303659
DRF_1_AutoML_20190324_043515,0.625375,2.66705,0.504083,0.2541
XGBoost_grid_1_AutoML_20190324_043515_model_2,0.635135,0.766937,0.500953,0.250954
GBM_3_AutoML_20190324_043515,0.637888,0.770484,0.495659,0.245677
GBM_2_AutoML_20190324_043515,0.637888,0.774643,0.495814,0.245832
GBM_4_AutoML_20190324_043515,0.637888,0.770856,0.495689,0.245708


In [72]:
predictions = aml.leader.predict(valid)

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [73]:
predictions

predict,negative,neutral,positive
positive,0.00575182,0.4516,0.542648
neutral,0.0592196,0.845148,0.0956328
neutral,0.003003,0.634395,0.362602
neutral,0.0165071,0.808507,0.174986
neutral,0.0309314,0.715505,0.253564
neutral,0.00567935,0.653404,0.340916
neutral,0.00878172,0.876626,0.114593
neutral,0.00857502,0.812669,0.178756
neutral,0.0650504,0.53822,0.39673
neutral,0.000842169,0.824635,0.174523


In [74]:
metrics = aml.leader.model_performance(valid)
metrics


ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.18912272620316287
RMSE: 0.43488242802298055
LogLoss: 0.6416816868769595
Mean Per-Class Error: 0.6185185185185186
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



negative,neutral,positive,Error,Rate
0.0,1.0,0.0,1.0,1 / 1
0.0,17.0,1.0,0.0555556,1 / 18
0.0,4.0,1.0,0.8,4 / 5
0.0,22.0,2.0,0.25,6 / 24


Top-3 Hit Ratios: 


k,hit_ratio
1,0.75
2,0.9583333
3,1.0


In [75]:
metrics.confusion_matrix()

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



negative,neutral,positive,Error,Rate
0.0,1.0,0.0,1.0,1 / 1
0.0,17.0,1.0,0.0555556,1 / 18
0.0,4.0,1.0,0.8,4 / 5
0.0,22.0,2.0,0.25,6 / 24
